# 任务 2b：抽象文本摘要

在此笔记本中，您将应对大型文档摘要中出现的挑战：输入文本超过模型上下文长度、生成幻觉输出或触发内存不足错误。

为缓解这些问题，此笔记本演示了一种使用提示词分块并通过 [LangChain](https://python.langchain.com/docs/get_started/introduction.html) 框架（一个支持应用程序利用语言模型的工具包）进行链接的架构。

您将探索一种方法来解决用户文档超过 Token 数限制的情况。分块会将文档拆分为多个低于上下文长度阈值的段，然后按顺序将其馈送给模型。这会将各个块的提示词链接在一起，从而保留先前的上下文。您可以应用这种方法来为通话记录、会议记录、书籍、文章、博客文章和其他相关内容生成摘要。

## 任务 2b.1：环境设置

在此任务中，您将设置环境。

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## 任务 2b.2：为长文本生成摘要 

### 配置 LangChain 以与 Boto3 配合使用

在此任务中，您需要为 LangChain Bedrock 类指定 LLM，并且可以传递用于推理的参数。

In [ ]:
# model configuration
from langchain_aws import BedrockLLM
modelId = "amazon.titan-text-premier-v1:0"
llm = BedrockLLM(
    model_id=modelId,
    model_kwargs={
        "maxTokenCount": 2048,
        "temperature": 0,
        "topP": 1
    },
    client=bedrock_client
)

## 任务 2b.3：加载包含许多 Token 的文本文件

在此任务中，您可以在“letters”目录中找到 [2022 年 Amazon CEO 致股东的信](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders) 的文本文件。以下单元会加载该文本文件并计算 Token 数。您将看到一条警告，其中指出该文本文件中的 Token 数超过了此模型的 Token 数上限。

In [ ]:
#get tokens
shareholder_letter = "../letters/2022-letter.txt"

with open(shareholder_letter, "r") as file:
    letter = file.read()
    
llm.get_num_tokens(letter)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i>**注意**：您可以放心地忽略这些警告，并继续执行下一个单元。

## 任务 2b.4：将长文本拆分成块

在此任务中，由于文本长度超过提示词的允许长度，您需要将文本拆分成较小的块。LangChain 中的 `RecursiveCharacterTextSplitter` 支持以递归方式将长文本拆分成块，直到每个块的大小均小于 `chunk_size`。使用 `separators=["\n\n", "\n"]` 将文本拆分成块，这样可避免将每个段落都拆分成多个块。

通过使每个块不超过 6,000 个字符，您可以分别获得各部分的摘要。一个块中的 Token 数或单词片段数量取决于文本。

In [ ]:
#chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n"], chunk_size=4000, chunk_overlap=100
)

docs = text_splitter.create_documents([letter])

In [ ]:
num_docs = len(docs)

num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

print(
    f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens"
)

## 任务 2b.5：为块生成摘要并进行合并

在此任务中，假设其他文档中的 Token 数一致，您就可以开始了。您可以使用 LangChain 的 `load_summarize_chain` 为文本生成摘要。`load_summarize_chain` 会提供三种生成摘要的方法：`stuff`、`map_reduce` 和 `refine`。

- `stuff`：将所有块放在一个提示词中。因此，这将达到 Token 数上限。
- `map_reduce`：为每个块生成摘要，合并摘要，然后为合并后的摘要生成摘要。如果合并后的摘要太大，则会报错。
- `refine`：为第一个块生成摘要，然后为第二个块和第一个摘要生成摘要。该过程会一直重复，直到为所有块都生成了摘要。

map_reduce 和 refine 均会多次调用 LLM，因此需要一些时间才能获得最终摘要。您可以在此处尝试 map_reduce。

In [ ]:
# Set verbose=True if you want to see the prompts being used
from langchain.chains.summarize import load_summarize_chain
summary_chain = load_summarize_chain(llm=llm, chain_type="map_reduce", verbose=False)

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i>**注意**：根据您的文档数量、Bedrock 请求速率配额和配置的重试设置，下方的链接过程可能需要一些时间来运行。

In [ ]:
#invoke chain
output = ""
try:
    
    output = summary_chain.invoke(docs)

except ValueError as error:
    if  "AccessDeniedException" in str(error):
        print(f"\x1b[41m{error}\
        \nTo troubeshoot this issue please refer to the following resources.\
         \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
         \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")      
        class StopExecution(ValueError):
            def _render_traceback_(self):
                pass
        raise StopExecution        
    else:
        raise error

In [ ]:
# print output
print(output['output_text'])

现在，您已尝试使用了提示词分块并通过 Langchain 框架进行链接来为大型文档生成摘要，同时缓解了因长输入文本而产生的问题。

### 自行尝试
- 将提示词更改为您的特定使用案例，并评估不同模型的输出。
- 尝试不同的 Token 长度，了解服务的延迟和响应能力。
- 应用不同的提示词工程原则，获得更好的输出。

### 清理

您已完成此笔记本。要进入本实验的下一部分，请执行以下操作：

- 关闭此笔记本文件并继续执行**任务 3**。